# Minio配置
1. 修改docker-compose.yaml中的用户名和密码，控制台输入 `docker-compose up -d` 运行后台minio.
2. 登录`http://localhost:9001/`
3. 创建bucket `finance-tiledb`
4. 创建access keys，并复制粘贴在以下代码
5. 依次执行以下代码初始化数据库

In [1]:
import tiledb

print("tiledb python version:", tiledb.__version__)
print("libtiledb core version:", tiledb.libtiledb.version())

tiledb python version: 0.31.1
libtiledb core version: (2, 25, 0)


In [2]:
config = tiledb.Config()

# Set configuration parameters
config["vfs.s3.scheme"] = "http"
config["vfs.s3.region"] = "" # timezone
config["vfs.s3.endpoint_override"] = "localhost:9000"
config["vfs.s3.use_virtual_addressing"] = "false"
# TODO 换成自己的ACCESS_KEY
config["vfs.s3.aws_access_key_id"] = "5CUUwM6cxHcg6MYvme4O" # 5AZvbYF8JH59jeZbOiFF
# TODO 换成自己的SECRET
config["vfs.s3.aws_secret_access_key"] = "mlNQJ9weN93PdSKYbzwSNycIX4RiY9urNgOiNB9H" # mOImsL4LuPuNg7yz43JXnTOi5uieK5NBEKeRj3QK

# Create contex object
ctx = tiledb.Ctx(config)

minio_bucket = "s3://finance-tiledb"

# 中国A股市场

In [3]:
# delete group
# my_group = f"{minio_bucket}/my_group"
# mygrp = tiledb.Group(my_group, "m", ctx=ctx)
# mygrp.delete()

cn_stock_group_name = f"{minio_bucket}/cn_stock"
try:
    tiledb.group_create(cn_stock_group_name, ctx=ctx)
except Exception as e:
    print(e)

S3: Error while listing with prefix 's3://finance-tiledb/cn_stock/__group/' and delimiter '/'[Error Type: 23] [HTTP Response Code: 403] [Exception: InvalidAccessKeyId] [Headers: 'accept-ranges' = 'bytes' 'content-length' = '360' 'content-type' = 'application/xml' 'date' = 'Mon, 19 Aug 2024 01:03:38 GMT' 'server' = 'MinIO' 'strict-transport-security' = 'max-age=31536000; includeSubDomains' 'vary' = 'Origin' 'x-amz-id-2' = 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8' 'x-amz-request-id' = '17ECFB9F1D1998DC' 'x-content-type-options' = 'nosniff' 'x-ratelimit-limit' = '1189' 'x-ratelimit-remaining' = '1189' 'x-xss-protection' = '1; mode=block'] : The Access Key Id you provided does not exist in our records.


## 交易日历 trade_cal

|字段名|说明|类型|
|--|--|--|
|dt [index] |日期|datetime64[s]|
|is_open|是否交易：0-不交易 1-交易|int8|

In [32]:
import numpy as np
import pandas as pd
array_trade_cal = f"{cn_stock_group_name}/trade_cal"

df = pd.read_csv("datasources/trade_cal.csv", parse_dates=['dt'], date_format='%Y%m%d')
# print(df)
# print(df.info())

# delete array_trade_cal
try:
    tiledb.Array.delete_array(array_trade_cal, ctx=ctx)
except:
    pass

d1 = tiledb.Dim(name="dt", domain=(df['dt'].iloc[0].to_datetime64().astype("datetime64[D]"), df['dt'].iloc[-1].to_datetime64().astype("datetime64[D]")), dtype="datetime64[D]")
dom1 = tiledb.Domain(d1)
a = tiledb.Attr(name="is_open", dtype=np.int8)
schema = tiledb.ArraySchema(domain=dom1, sparse=False, attrs=[a], cell_order='col-major', tile_order='col-major')
schema.check()
tiledb.Array.create(array_trade_cal, schema, ctx=ctx)

with tiledb.open(array_trade_cal, mode='w', ctx=ctx) as A:
    A[:] = df['is_open'].to_numpy()
schema

              dt  is_open
0     1990-12-19        1
1     1990-12-20        1
2     1990-12-21        1
3     1990-12-22        0
4     1990-12-23        0
...          ...      ...
12427 2024-12-27        1
12428 2024-12-28        0
12429 2024-12-29        0
12430 2024-12-30        1
12431 2024-12-31        1

[12432 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12432 entries, 0 to 12431
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   dt       12432 non-null  datetime64[ns]
 1   is_open  12432 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 194.4 KB
None
ArraySchema(
  domain=Domain(*[
    Dim(name='dt', domain=(numpy.datetime64('1990-12-19'), numpy.datetime64('2024-12-31')), tile=numpy.timedelta64(12432,'D'), dtype='datetime64[D]'),
  ]),
  attrs=[
    Attr(name='is_open', dtype='int8', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-majo

## 证券基本信息 basic_info_stocks

|字段名|说明|类型|
|--|--|--|
|symbol|股票代码|string|
|name|股票名称|string|
|area|地域|string|
|industry|所属行业|string|
|fullname|股票全称|string|
|enname|英文全称|string|
|cnspell|拼音缩写|string|
|market|市场类型（主板/创业板/科创板/CDR）|string|
|exchange|交易所代码（SSE/SZSE/BSE）|string|
|curr_type|交易货币：CNY|string|
|list_date|上市日期|datetime64[D]|
|delist_date|退市日期|datetime64[D]|
|status|上市状态，其中L上市 D退市 P暂停上市|string|
|is_hs|是否沪深港通标的，N否 H沪股通 S深股通|string|

In [6]:
import numpy as np
import pandas as pd
array_basic_info_stocks = f"{cn_stock_group_name}/basic_info_stocks"

df = pd.read_csv("datasources/basic_info_stocks.csv", parse_dates=['list_date', 'delist_date'], date_format='%Y%m%d', index_col=0)
df = df.drop(columns=['symbol'])
df = df.rename(columns={'ts_code':'symbol', 'list_status':'status'})
df = df.fillna('')
# print(df)
# print(df.info())

# delete array_basic_info_stocks
try:
    tiledb.Array.delete_array(array_basic_info_stocks, ctx=ctx)
except:
    pass

# TODO add zip filter or category type
attrs = [
    tiledb.Attr(name="symbol", dtype=np.dtype('U')),
    tiledb.Attr(name="name", dtype=np.dtype('U')),
    tiledb.Attr(name="area", dtype=np.dtype('U')),
    tiledb.Attr(name="industry", dtype=np.dtype('U')),
    tiledb.Attr(name="fullname", dtype=np.dtype('U')),
    tiledb.Attr(name="enname", dtype=np.dtype('U')),
    tiledb.Attr(name="cnspell", dtype=np.dtype('U')),
    tiledb.Attr(name="market", dtype=np.dtype('U')),
    tiledb.Attr(name="exchange", dtype=np.dtype('U')),
    tiledb.Attr(name="curr_type", dtype=np.dtype('U')),
    tiledb.Attr(name="list_date", dtype="datetime64[D]"),
    tiledb.Attr(name="delist_date", dtype="datetime64[D]", nullable=True),
    tiledb.Attr(name="status", dtype=np.dtype('U')),
    tiledb.Attr(name="is_hs", dtype=np.dtype('U')),
]
d1 = tiledb.Dim(domain=(0,len(df)-1), dtype=np.uint32)
dom1 = tiledb.Domain(d1)
schema = tiledb.ArraySchema(domain=dom1, sparse=False, attrs=attrs, cell_order='col-major', tile_order='col-major')
schema.check()
tiledb.Array.create(array_basic_info_stocks, schema, ctx=ctx)

with tiledb.open(array_basic_info_stocks, mode='w', ctx=ctx) as A:
    A[:] = df.to_dict('series')
schema

        symbol     name area industry          fullname  \
0    000001.SZ     平安银行   深圳       银行        平安银行股份有限公司   
1    000002.SZ      万科A   深圳     全国地产        万科企业股份有限公司   
2    000004.SZ     国华网安   深圳     软件服务    深圳国华网安科技股份有限公司   
3    000006.SZ     深振业A   深圳     区域地产   深圳市振业(集团)股份有限公司   
4    000007.SZ      全新好   深圳     其他商业      深圳市全新好股份有限公司   
..         ...      ...  ...      ...               ...   
270  688555.SH   退泽达(退)                泽达易盛(天津)科技股份有限公司   
271  832317.BJ  观典防务(退)                    观典防务技术股份有限公司   
272  833874.BJ  泰祥股份(退)                   十堰市泰祥实业股份有限公司   
273  833994.BJ  翰博高新(退)                翰博高新材料(合肥)股份有限公司   
274  T00018.SH  上港集箱(退)                    上海港集装箱股份有限公司   

                                                enname cnspell market  \
0                               Ping An Bank Co., Ltd.    payh     主板   
1                                 China Vanke Co.,Ltd.     wka     主板   
2    Shenzhen Guohua Network Security Technology Co...    ghwa     主板   


## 日线详情数据 bars/daily

|字段名|说明|类型|
|--|--|--|
|dt [index]|交易日期|datetime64[s]|
|name|股票名称|string|
|open|开盘价(前复权)|float32|
|high|最高价(前复权)|float32|
|low|最低价(前复权)|float32|
|close|收盘价(前复权)|float32|
|_open|开盘价|float32|
|_high|最高价|float32|
|_low|最低价|float32|
|_close|收盘价|float32|
|preclose|前收盘价，若当天发生除权，前收盘价为上个交易日复权之后的收盘价|float32|
|volume|成交量（股）|uint32|
|amount|成交额（元）|uint64|
|net_profit_ttm|净利润TTM(不含少数股东损益)|float64|
|cashflow_ttm|现金流TTM, 经营活动产生的现金流量净额|float64|
|equity|净资产|float64|
|asset|总资产|float64|
|debt|总负债|float64|
|debttoasset|资产负债率|float32|
|net_profit_q|净利润(当季)|float64|
|pe_ttm|滚动市盈率|float32|
|pb|市净率|float32|
|mkt_cap|总市值(元)|float64|
|mkt_cap_ashare|流通市值(元)|float64|
|vip_buy_amt|大户资金买入额(万元)|float32|
|vip_sell_amt|大户资金卖出额(万元)|float32|
|inst_buy_amt|机构资金买入额(万元)|float32|
|inst_sell_amt|机构资金卖出额(万元)|float32|
|mid_buy_amt|中户资金买入额(万元)|float32|
|mid_sell_amt|中户资金卖出额(万元)|float32|
|indi_buy_amt|散户资金买入额(万元)|float32|
|indi_sell_amt|散户资金卖出额(万元)|float32|
|master_net_flow_in|主力(机构和大户)净买入(万元)|float32|
|master2_net_flow_in|主力2(机构、大户和中户)净买入(万元)|float32|
|vip_net_flow_in|大户净流入(万元)|float32|
|mid_net_flow_in|中户净流入(万元)|float32|
|inst_net_flow_in|机构净流入(万元)|float32|
|indi_net_flow_in|散户净流入(万元)|float32|
|total_sell_amt|流出资金总额(万元)|float32|
|total_buy_amt|流入资金总额(万元)|float32|
|net_flow_in|资金净流入(万元)|float32|
|turnover|换手率|float32|
|free_shares|流通股本|uint64|
|total_shares|总股本|uint64|
|maxupordown|标记收盘涨停或跌停状态,1-涨停,2-一字板涨停；-1-跌停，-2-一字板跌停；0-未涨跌停|uint8|
|maxupordown_at_open|标记开盘涨停或跌停状态，状态码同上|uint8|
|lb_up_count|连板涨停次数|uint8|
|lb_down_count|连板跌停次数|uint8|


In [48]:
import numpy as np
array_bars_stock_daily = f"{cn_stock_group_name}/bars/daily"

# delete array_bars_stock_daily
try:
    tiledb.Array.delete_array(array_bars_stock_daily, ctx=ctx)
except:
    pass

# filter_list = tiledb.FilterList([tiledb.ByteShuffleFilter(), tiledb.ZstdFilter()])
attrs = [
    tiledb.Attr(name="name", dtype='S16'),
    tiledb.Attr(name="open", dtype=np.float32),
    tiledb.Attr(name="high", dtype=np.float32),
    tiledb.Attr(name="low", dtype=np.float32),
    tiledb.Attr(name="close", dtype=np.float32),
    tiledb.Attr(name="_open", dtype=np.float32),
    tiledb.Attr(name="_high", dtype=np.float32),
    tiledb.Attr(name="_low", dtype=np.float32),
    tiledb.Attr(name="_close", dtype=np.float32),
    tiledb.Attr(name="volume", dtype=np.uint32),
    tiledb.Attr(name="amount", dtype=np.uint64),
    tiledb.Attr(name="preclose", dtype=np.float32),

    tiledb.Attr(name="net_profit_ttm", dtype=np.float64),
    tiledb.Attr(name="cashflow_ttm", dtype=np.float64),
    tiledb.Attr(name="equity", dtype=np.float64),
    tiledb.Attr(name="asset", dtype=np.float64),
    tiledb.Attr(name="debt", dtype=np.float64),
    tiledb.Attr(name="debttoasset", dtype=np.float32),
    tiledb.Attr(name="net_profit_q", dtype=np.float64),
    tiledb.Attr(name="pe_ttm", dtype=np.float32),
    tiledb.Attr(name="pb", dtype=np.float32),
    tiledb.Attr(name="mkt_cap", dtype=np.float64),
    tiledb.Attr(name="mkt_cap_ashare", dtype=np.float64),

    tiledb.Attr(name="vip_buy_amt", dtype=np.float32),
    tiledb.Attr(name="vip_sell_amt", dtype=np.float32),
    tiledb.Attr(name="inst_buy_amt", dtype=np.float32),
    tiledb.Attr(name="inst_sell_amt", dtype=np.float32),
    tiledb.Attr(name="mid_buy_amt", dtype=np.float32),
    tiledb.Attr(name="mid_sell_amt", dtype=np.float32),
    tiledb.Attr(name="indi_buy_amt", dtype=np.float32),
    tiledb.Attr(name="indi_sell_amt", dtype=np.float32),
    tiledb.Attr(name="master_net_flow_in", dtype=np.float32),
    tiledb.Attr(name="master2_net_flow_in", dtype=np.float32),
    tiledb.Attr(name="vip_net_flow_in", dtype=np.float32),
    tiledb.Attr(name="mid_net_flow_in", dtype=np.float32),
    tiledb.Attr(name="inst_net_flow_in", dtype=np.float32),
    tiledb.Attr(name="indi_net_flow_in", dtype=np.float32),
    tiledb.Attr(name="total_sell_amt", dtype=np.float32),
    tiledb.Attr(name="total_buy_amt", dtype=np.float32),
    tiledb.Attr(name="net_flow_in", dtype=np.float32),

    tiledb.Attr(name="turnover", dtype=np.float32),
    tiledb.Attr(name="free_shares", dtype=np.uint64),
    tiledb.Attr(name="total_shares", dtype=np.uint64),
    tiledb.Attr(name="maxupordown", dtype=np.uint8),
    tiledb.Attr(name="maxupordown_at_open", dtype=np.uint8),
    tiledb.Attr(name="lb_up_count", dtype=np.uint8),
    tiledb.Attr(name="lb_down_count", dtype=np.uint8),
]
d1 = tiledb.Dim(name="_dt", domain=(0, 25000), tile=250, dtype=np.uint32)
d2 = tiledb.Dim(name='_symbol', domain=(0,20000), tile=1, dtype=np.uint32)
dom1 = tiledb.Domain(d1, d2)
dim_labels = {
    0: {
        "dt": d1.create_label_schema("increasing", "datetime64[s]")
    },
    1: {
        "symbol": d2.create_label_schema("increasing", 'S9')
    }
}
schema = tiledb.ArraySchema(domain=dom1, sparse=False, attrs=attrs, cell_order='col-major', tile_order='col-major', dim_labels=dim_labels)
schema.check()
tiledb.Array.create(array_bars_stock_daily, schema, ctx=ctx)
schema

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 25000), tile=250, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='name', dtype='|S16', var=False, nullable=False, enum_label=None),
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='_open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='_high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='_low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='_close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint32', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='preclose', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='net_profit_ttm', dtype='float64', var=False, nullable=False, enum_label=None),
    Attr(name='cashflow_ttm', dtype='float64', var=False, nullable=False, enum_label=None),
    Attr(name='equity', dtype='float64', var=False, nullable=False, enum_label=None),
    Attr(name='asset', dtype='float64', var=False, nullable=False, enum_label=None),
    Attr(name='debt', dtype='float64', var=False, nullable=False, enum_label=None),
    Attr(name='debttoasset', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='net_profit_q', dtype='float64', var=False, nullable=False, enum_label=None),
    Attr(name='pe_ttm', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='pb', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='mkt_cap', dtype='float64', var=False, nullable=False, enum_label=None),
    Attr(name='mkt_cap_ashare', dtype='float64', var=False, nullable=False, enum_label=None),
    Attr(name='vip_buy_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='vip_sell_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='inst_buy_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='inst_sell_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='mid_buy_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='mid_sell_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='indi_buy_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='indi_sell_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='master_net_flow_in', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='master2_net_flow_in', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='vip_net_flow_in', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='mid_net_flow_in', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='inst_net_flow_in', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='indi_net_flow_in', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='total_sell_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='total_buy_amt', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='net_flow_in', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='turnover', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='free_shares', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='total_shares', dtype='uint64', var=False, nu

## K线数据（分钟线、小时线、周线、月线）bars

已经是前复权的数据

- 分钟线 bars/1min(5min/10min/...)
- 小时线 bars/1h(2h)
- 周线 bars/weekly
- 月线 bars/monthly

|字段名|说明|类型|
|--|--|--|
|dt|交易日期|datetime64[s]|
|open|开盘价|float32|
|high|最高价|float32|
|low|最低价|float32|
|close|收盘价|float32|
|volume|成交量|uint64|
|amount|成交额|float64|

In [32]:
import numpy as np
# 都按100年来估计
def setup_bars(_name, _d1_domain, _d1_tile):
    array_bars = f"{cn_stock_group_name}/bars/{_name}"

    # delete array_bars
    try:
        tiledb.Array.delete_array(array_bars, ctx=ctx)
    except:
        pass

    # filter_list = tiledb.FilterList([tiledb.ByteShuffleFilter(), tiledb.ZstdFilter()])
    attrs = [
        tiledb.Attr(name="open", dtype=np.float32),
        tiledb.Attr(name="high", dtype=np.float32),
        tiledb.Attr(name="low", dtype=np.float32),
        tiledb.Attr(name="close", dtype=np.float32),
        tiledb.Attr(name="volume", dtype=np.uint64),
        tiledb.Attr(name="amount", dtype=np.float64),
    ]
    d1 = tiledb.Dim(name="_dt", domain=_d1_domain, tile=_d1_tile, dtype=np.uint32)
    d2 = tiledb.Dim(name='_symbol', domain=(0,20000), tile=1, dtype=np.uint32)
    dom1 = tiledb.Domain(d1, d2)
    dim_labels = {
        0: {
            "dt": d1.create_label_schema("increasing", "datetime64[s]")
        },
        1: {
            "symbol": d2.create_label_schema("increasing", np.dtype('U'))
        }
    }
    schema = tiledb.ArraySchema(domain=dom1, sparse=False, attrs=attrs, cell_order='col-major', tile_order='col-major', dim_labels=dim_labels)
    schema.check()
    tiledb.Array.create(array_bars, schema, ctx=ctx)
    return schema

In [34]:
setup_bars("1min", (0, 240*250*100), 1000)

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 6000000), tile=1000, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

In [35]:
setup_bars("5min", (0, 48*250*100), 1000)

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 1200000), tile=1000, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

In [36]:
setup_bars("15min", (0, 16*250*100), 1000)

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 400000), tile=1000, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

In [37]:
setup_bars("30min", (0, 8*250*100), 1000)

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 200000), tile=1000, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

In [38]:
setup_bars("1h", (0, 4*250*100), 1000)

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 100000), tile=1000, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

In [39]:
setup_bars("2h", (0, 2*250*100), 1000)

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 50000), tile=1000, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

In [40]:
setup_bars("weekly", (0, 6000), None)

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 6000), tile=6001, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

In [41]:
setup_bars("monthly", (0, 1200), None)

ArraySchema(
  domain=Domain(*[
    Dim(name='_dt', domain=(0, 1200), tile=1201, dtype='uint32'),
    Dim(name='_symbol', domain=(0, 20000), tile=1, dtype='uint32'),
  ]),
  attrs=[
    Attr(name='open', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='high', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='low', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='close', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='volume', dtype='uint64', var=False, nullable=False, enum_label=None),
    Attr(name='amount', dtype='float64', var=False, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

## 每日统计数据 market_stats

排除了ST股

|字段名|说明|类型|
|--|--|--|
|dt [index]|交易日期|datetime64[s]|
|count_of_uplimit|涨停数|uint16|
|count_of_downlimit|跌停数|uint16|
|count_of_yiziup|一字涨停数|uint16|
|count_of_yizidown|一字跌停数|uint16|
|ratio_of_uplimit|涨停数占所有股数比例|float32|
|ratio_of_downlimit|跌停数占所有股数比例|float32|
|ratio_of_yiziup|一字涨停数占所有股数比例|float32|
|ratio_of_yizidown|一字跌停数占所有股数比例|float32|
|lb|连板数量[首板数, 二板数, ...]|[uint8,...]|


In [42]:

import numpy as np
array_market_stats = f"{cn_stock_group_name}/market_stats"

# delete array_market_stats
try:
    tiledb.Array.delete_array(array_market_stats, ctx=ctx)
except:
    pass

attrs = [
    tiledb.Attr(name="count_of_uplimit", dtype=np.uint16),
    tiledb.Attr(name="count_of_downlimit", dtype=np.uint16),
    tiledb.Attr(name="count_of_yiziup", dtype=np.uint16),
    tiledb.Attr(name="count_of_yizidown", dtype=np.uint16),
    tiledb.Attr(name="ratio_of_uplimit", dtype=np.float32),
    tiledb.Attr(name="ratio_of_downlimit", dtype=np.float32),
    tiledb.Attr(name="ratio_of_yiziup", dtype=np.float32),
    tiledb.Attr(name="ratio_of_yizidown", dtype=np.float32),
    tiledb.Attr(name="lb", var=True, dtype=np.uint8),
]
d1 = tiledb.Dim(name="dt", domain=(np.datetime64('1990-01-01'), np.datetime64('2100-01-01')), tile=250, dtype="datetime64[s]")
dom1 = tiledb.Domain(d1)
schema = tiledb.ArraySchema(domain=dom1, sparse=False, attrs=attrs, cell_order='col-major', tile_order='col-major')
schema.check()
tiledb.Array.create(array_market_stats, schema, ctx=ctx)
schema


ArraySchema(
  domain=Domain(*[
    Dim(name='dt', domain=(numpy.datetime64('1990-01-01T00:00:00'), numpy.datetime64('2100-01-01T00:00:00')), tile=numpy.timedelta64(250,'s'), dtype='datetime64[s]'),
  ]),
  attrs=[
    Attr(name='count_of_uplimit', dtype='uint16', var=False, nullable=False, enum_label=None),
    Attr(name='count_of_downlimit', dtype='uint16', var=False, nullable=False, enum_label=None),
    Attr(name='count_of_yiziup', dtype='uint16', var=False, nullable=False, enum_label=None),
    Attr(name='count_of_yizidown', dtype='uint16', var=False, nullable=False, enum_label=None),
    Attr(name='ratio_of_uplimit', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='ratio_of_downlimit', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='ratio_of_yiziup', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='ratio_of_yizidown', dtype='float32', var=False, nullable=False, enum_label=None),
    Attr(name='lb', dtype='uint8', var=True, nullable=False, enum_label=None),
  ],
  cell_order='col-major',
  tile_order='col-major',
  sparse=False,
)

# 中国期货市场

In [43]:
cn_future_group_name = f"{minio_bucket}/cn_future"
tiledb.group_create(cn_future_group_name, ctx=ctx)